# SimCLR
PyTorch implementation of SimCLR: A Simple Framework for Contrastive Learning of Visual Representations by T. Chen et al. With support for the LARS (Layer-wise Adaptive Rate Scaling) optimizer.

[Link to paper](https://arxiv.org/pdf/2002.05709.pdf)


## Setup the repository

In [1]:
#!git clone https://github.com/AmeerHamza111/SimCLR.git
%cd SimCLR
#!wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar
#!sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
#!pip install  pyyaml --upgrade

/scratch/sga297/dlProject/DLFrnn/Frnn/SimClr/SimclrGPU/SimCLR


# Part 1:
## SimCLR pre-training

In [2]:
# whether to use a TPU or not (set in Runtime -> Change Runtime Type)
use_tpu = False

#### Install PyTorch/XLA

In [3]:
if use_tpu:
  VERSION = "20200220" #@param ["20200220","nightly", "xrt==1.15.0"]
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION

In [4]:
import os
import torch

if use_tpu:
  # imports the torch_xla package for TPU support
  import torch_xla
  import torch_xla.core.xla_model as xm
  dev = xm.xla_device()
  print(dev)
  
import torchvision
import argparse

from torch.utils.tensorboard import SummaryWriter

apex = False
try:
    from apex import amp
    apex = True
except ImportError:
    print(
        "Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training"
    )

from model import load_model, save_model
from modules import NT_Xent
from modules.transformations import TransformsSimCLR
from utils import mask_correlated_samples, post_config_hook


Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training


In [5]:
def train(args, train_loader, model, criterion, optimizer, writer):
    loss_epoch = 0
    for step, ((x_i, x_j), _) in enumerate(train_loader):

        optimizer.zero_grad()
        x_i = x_i.to(args.device)
        x_j = x_j.to(args.device)

        # positive pair, with encoding
        h_i, z_i = model(x_i)
        h_j, z_j = model(x_j)

        loss = criterion(z_i, z_j)

        if apex and args.fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

        optimizer.step()

        if step % 50 == 0:
            print(f"Step [{step}/{len(train_loader)}]\t Loss: {loss.item()}")

        torch.cuda.empty_cache()
        writer.add_scalar("Loss/train_epoch", loss.item(), args.global_step)
        loss_epoch += loss.item()
        args.global_step += 1

    return loss_epoch

### Load arguments from `config/config.yaml`

In [6]:
from pprint import pprint
from utils.yaml_config_hook import yaml_config_hook

config = yaml_config_hook("./config/config.yaml")
args = argparse.Namespace(**config)

if use_tpu:
  args.device = dev
else:
  args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
args.out_dir = "logs/simclrPretext"
if not os.path.exists("logs/simclrPretext"):
  os.makedirs("logs/simclrPretext")

In [7]:
args.batch_size = 2
args.epochs = 1
args.epoch_num = 1
args.resnet = "resnet18"
args.dataset = "road"
args.model_path = "logs/simclrPretext"
args.optimizer = "LARS"
pprint(vars(args))

{'batch_size': 2,
 'dataset': 'road',
 'device': device(type='cuda', index=0),
 'epoch_num': 1,
 'epochs': 1,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 100,
 'model_path': 'logs/simclrPretext',
 'normalize': True,
 'optimizer': 'LARS',
 'out_dir': 'logs/simclrPretext',
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [8]:
image_folder = 'data'
annotation_csv = 'data/annotation.csv'

In [9]:
import numpy as np
from data_helper import SimclrUnlabeledDataset
from helper import convert_map_to_lane_map, convert_map_to_road_map, collate_fn, draw_box

In [10]:
unlabeled_scene_index = np.arange(106)

### Load dataset into train loader

In [11]:
root = "./datasets"

train_sampler = None

if args.dataset == "STL10":
    train_dataset = torchvision.datasets.STL10(
        root, split="unlabeled", download=True, transform=TransformsSimCLR()
    )
elif args.dataset == "CIFAR10":
    train_dataset = torchvision.datasets.CIFAR10(
        root, download=True, transform=TransformsSimCLR()
    )
elif args.dataset == "road":
    train_dataset = SimclrUnlabeledDataset(image_folder=image_folder, 
      scene_index=unlabeled_scene_index, first_dim='sample', transform=TransformsSimCLR())
else:
    raise NotImplementedError

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    shuffle=(train_sampler is None),
    drop_last=True,
    num_workers=args.workers,
    sampler=train_sampler,
)

In [12]:
#--NotebookApp.iopub_msg_rate_limit = 4000.0

### Load the SimCLR model, optimizer and learning rate scheduler

In [13]:
model, optimizer, scheduler = load_model(args, train_loader)

Let's use 2 GPUs!


In [14]:
print(model)

DataParallel(
  (module): SimCLR(
    (encoder): ResNet(
      (conv1): Conv2d(18, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): Batch

### Setup TensorBoard for logging experiments

In [15]:
tb_dir = os.path.join(args.out_dir, "colab")
if not os.path.exists(tb_dir):
  os.makedirs(tb_dir)
writer = SummaryWriter(log_dir=tb_dir)

### Create the mask that will remove correlated samples from the negative examples

In [16]:
mask = mask_correlated_samples(args)

### Initialize the criterion (NT-Xent loss)

In [17]:
criterion = NT_Xent(args.batch_size, args.temperature, mask, args.device)

### Start training

In [18]:
args.global_step = 0
args.current_epoch = 0
for epoch in range(args.start_epoch, args.epochs):
    lr = optimizer.param_groups[0]['lr']
    loss_epoch = train(args, train_loader, model, criterion, optimizer, writer)

    if scheduler:
        scheduler.step()

    if epoch % 1 == 0:
        save_model(args, model, optimizer)

    writer.add_scalar("Loss/train", loss_epoch / len(train_loader), epoch)
    writer.add_scalar("Misc/learning_rate", lr, epoch)
    print(
        f"Epoch [{epoch}/{args.epochs}]\t Loss: {loss_epoch / len(train_loader)}\t lr: {round(lr, 5)}"
    )
    args.current_epoch += 1

## end training
save_model(args, model, optimizer)

Step [0/6678]	 Loss: 1.0985398292541504
Step [50/6678]	 Loss: 1.0992794036865234
Step [100/6678]	 Loss: 1.0991476774215698
Step [150/6678]	 Loss: 1.0981806516647339
Step [200/6678]	 Loss: 1.0985594987869263
Step [250/6678]	 Loss: 1.0986725091934204
Step [300/6678]	 Loss: 1.0987493991851807
Step [350/6678]	 Loss: 1.0988284349441528
Step [400/6678]	 Loss: 1.0991953611373901
Step [450/6678]	 Loss: 1.0980743169784546
Step [500/6678]	 Loss: 1.097842812538147
Step [550/6678]	 Loss: 1.0987294912338257
Step [600/6678]	 Loss: 1.0984667539596558
Step [650/6678]	 Loss: 1.0984159708023071
Step [700/6678]	 Loss: 1.0986155271530151
Step [750/6678]	 Loss: 1.0981274843215942
Step [800/6678]	 Loss: 1.0987215042114258
Step [850/6678]	 Loss: 1.099386215209961
Step [900/6678]	 Loss: 1.098129391670227
Step [950/6678]	 Loss: 1.099453091621399
Step [1000/6678]	 Loss: 1.098272442817688
Step [1050/6678]	 Loss: 1.098783016204834
Step [1100/6678]	 Loss: 1.0993229150772095
Step [1150/6678]	 Loss: 1.09869539737701

## Download last checkpoint to local drive (replace `100` with `args.epochs`)

In [19]:
#from google.colab import files
#files.download('./logs/checkpoint_'+ str(args.epochs) +'.tar')

# Part 2:
## Linear evaluation using logistic regression, using weights from frozen, pre-trained SimCLR model

In [20]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import argparse

from experiment import ex
from model import load_model
from utils import post_config_hook

#from modules import LogisticRegression


In [21]:
def compute_ts_road_map(road_map1, road_map2):
    tp = (road_map1 * road_map2).sum()

    return tp * 1.0 / (road_map1.sum() + road_map2.sum() - tp)

In [22]:
def train(args, loader, simclr_model, model,criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model.to("cuda")
    model.train()
    #with torch.no_grad():
    for step, (x, y) in enumerate(loader):
        #print(x.shape)
        #print(y.shape)
        y = y.type(torch.float)
        y = y.reshape(-1, 640000)
        optimizer.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)
        

        # get encoding
        with torch.no_grad():
            h, z = simclr_model(x)
            #k = model1(h)
            #h = 512
            #z = 64
            #print(h.shape)
            #print(z.shape)
        #loss =0
        #for i,j in zip(h,y):
            output = model(h)
            loss = criterion(output, y)
            loss.requires_grad = True

        #predicted = output.argmax(1)
        #acc = (predicted == y).sum().item() / y.size(0)
            iou = compute_ts_road_map(output,y)
            accuracy_epoch += iou
        #accuracy_epoch += acc

            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()
        if step % 1 == 0:
            print(f"Step [{step}/{len(loader)}]\t Loss: {loss.item()}\t Accuracy: {iou}")
        torch.cuda.empty_cache()

    return loss_epoch, accuracy_epoch

In [23]:
def test(args, loader, simclr_model, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model.to("cuda")
    model.eval()
    for step, (x, y) in enumerate(loader):
        model.zero_grad()
        y = y.type(torch.float)
        y = y.reshape(-1, 640000)

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h, z = simclr_model(x)
            # h = 512
            # z = 64

        
            output = model(h)
            loss = criterion(output, y)

            #predicted = output.argmax(1)
            #acc = (predicted == y).sum().item() / y.size(0)
            iou = compute_ts_road_map(output,y)
            accuracy_epoch += iou
            #accuracy_epoch += acc

            loss_epoch += loss.item()
        torch.cuda.empty_cache()


    return loss_epoch, accuracy_epoch

In [24]:
from pprint import pprint
from utils.yaml_config_hook import yaml_config_hook

config = yaml_config_hook("./config/config.yaml")
pprint(config)
args = argparse.Namespace(**config)

if use_tpu:
  args.device = dev
else:
  args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

{'batch_size': 128,
 'dataset': 'STL10',
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 100,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'projection_dim': 64,
 'resnet': 'resnet50',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [25]:
args.batch_size = 2
args.resnet = "resnet18"
args.model_path = "logs/simclrPretext"
args.epochs = 1
args.epoch_num = 1
args.dataset = 'road'
args.logistic_epochs =1
args.logistic_batch_size =4
pprint(vars(args))

{'batch_size': 2,
 'dataset': 'road',
 'device': device(type='cuda'),
 'epoch_num': 1,
 'epochs': 1,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 4,
 'logistic_epochs': 1,
 'model_path': 'logs/simclrPretext',
 'normalize': True,
 'optimizer': 'Adam',
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [26]:
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 120)

## validation scene index.
validation_scene_index = np.arange(120, 134)

In [27]:
train_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.RandomResizedCrop(size=(256,306)),
                torchvision.transforms.ToTensor(),
            ]
        )

In [28]:
from data_helper import SimclrLabeledDataset

### Load dataset into train/test dataloaders

In [29]:
root = "./datasets"
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

if args.dataset == "STL10":
    train_dataset = torchvision.datasets.STL10(
        root, split="train", download=True, transform=torchvision.transforms.ToTensor()
    )
    test_dataset = torchvision.datasets.STL10(
        root, split="test", download=True, transform=torchvision.transforms.ToTensor()
    )
elif args.dataset == "CIFAR10":
    train_dataset = torchvision.datasets.CIFAR10(
        root, train=True, download=True, transform=transform
    )
    test_dataset = torchvision.datasets.CIFAR10(
        root, train=False, download=True, transform=transform
    )
elif args.dataset == "road":
    train_dataset = SimclrLabeledDataset(image_folder=image_folder, 
      scene_index = labeled_scene_index, annotation_file=annotation_csv,transform = train_transform )
    test_dataset = SimclrLabeledDataset(image_folder=image_folder, 
      scene_index = validation_scene_index, annotation_file=annotation_csv, transform = train_transform)
else:
    raise NotImplementedError

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=args.workers,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=False,
    drop_last=True,
    num_workers=args.workers,
)

In [30]:
torch.cuda.get_device_properties(args.device).total_memory

11996954624

### Load SimCLR model and load model weights

In [31]:
simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
simclr_model = simclr_model.to(args.device)
simclr_model.eval()

Let's use 2 GPUs!


DataParallel(
  (module): SimCLR(
    (encoder): ResNet(
      (conv1): Conv2d(18, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): Batch

In [32]:
class ASPP(nn.Module):

    def __init__(self, C, depth, num_classes, conv=nn.Conv2d, norm=nn.BatchNorm2d, momentum=0.0003, mult=1):
        super(ASPP, self).__init__()
        self._C = C
        self._depth = depth
        self._num_classes = num_classes

        self.global_pooling = nn.AdaptiveAvgPool2d(1)
        self.relu = nn.ReLU(inplace=True)
        self.aspp1 = conv(C, depth, kernel_size=1, stride=1, bias=False)
        self.aspp2 = conv(C, depth, kernel_size=3, stride=1,
                               dilation=int(6*mult), padding=int(6*mult),
                               bias=False)
        self.aspp3 = conv(C, depth, kernel_size=3, stride=1,
                               dilation=int(12*mult), padding=int(12*mult),
                               bias=False)
        self.aspp4 = conv(C, depth, kernel_size=3, stride=1,
                               dilation=int(18*mult), padding=int(18*mult),
                               bias=False)
        self.aspp5 = conv(C, depth, kernel_size=1, stride=1, bias=False)
        self.aspp1_bn = norm(depth, momentum)
        self.aspp2_bn = norm(depth, momentum)
        self.aspp3_bn = norm(depth, momentum)
        self.aspp4_bn = norm(depth, momentum)
        self.aspp5_bn = norm(depth, momentum)
        self.conv2 = conv(depth * 5, depth, kernel_size=1, stride=1,
                               bias=False)
        self.bn2 = norm(depth, momentum)
        self.conv3 = nn.Conv2d(depth, num_classes, kernel_size=1, stride=1, bias=False)

    def forward(self, x):
        x1 = self.aspp1(x)
        print(x1.shape)
        x1 = self.aspp1_bn(x1)
        print(x1.shape)
        x1 = self.relu(x1)
        print(x1.shape)
        x2 = self.aspp2(x)
        print(x2.shape)
        x2 = self.aspp2_bn(x2)
        print(x2.shape)
        x2 = self.relu(x2)
        print(x2.shape)
        x3 = self.aspp3(x)
        print(x3.shape)
        x3 = self.aspp3_bn(x3)
        print(x3.shape)
        x3 = self.relu(x3)
        print(x3.shape)
        x4 = self.aspp4(x)
        print(x4.shape)
        x4 = self.aspp4_bn(x4)
        print(x4.shape)
        x4 = self.relu(x4)
        print(x4.shape)
        x5 = self.global_pooling(x)
        print(x5.shape)
        x5 = self.aspp5(x5)
        print(x5.shape)
        x5 = self.aspp5_bn(x5)
        print(x5.shape)
        x5 = self.relu(x5)
        x5 = nn.Upsample((x.shape[2], x.shape[3]), mode='bilinear',
                         align_corners=True)(x5)
        print(x5.shape)
        x = torch.cat((x1, x2, x3, x4, x5), 1)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.bn2(x)
        print(x.shape)
        x = self.relu(x)
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)
        print("*******")

        return x


In [33]:
pprint(vars(args))

{'batch_size': 2,
 'dataset': 'road',
 'device': device(type='cuda'),
 'epoch_num': 1,
 'epochs': 1,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 4,
 'logistic_epochs': 1,
 'model_path': 'logs/simclrPretext',
 'normalize': True,
 'optimizer': 'Adam',
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [34]:
#print(list(list(simclr_model.children())[0].children())[0])

In [35]:
#print(simclr_model.module.encoder)

In [36]:
#model1 = nn.Sequential(simclr_model.module.encoder.conv1, simclr_model.module.encoder.bn1, simclr_model.module.encoder.relu,
                                     #simclr_model.module.encoder.maxpool, simclr_model.module.encoder.layer1, simclr_model.module.encoder.layer2,
                                    #simclr_model.module.encoder.layer3, simclr_model.module.encoder.layer4)

#model1 = nn.DataParallel(model1)
#model1 = model1.to(args.device)
#simclr_model.eval()
#model1.eval()


In [37]:
#print(model1)

In [38]:
print(simclr_model)

DataParallel(
  (module): SimCLR(
    (encoder): ResNet(
      (conv1): Conv2d(18, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): Batch

In [39]:
import torch.nn as nn

class Reshape(nn.Module):
    def __init__(self):
        super(Reshape, self).__init__()
        #self.shape = [256,512,1,1]
    def forward(self, x):
        return x.view(64,512,2,2)

In [40]:
class Reshape1(nn.Module):
    def __init__(self):
        super(Reshape1, self).__init__()
        #self.shape = [256,512,1,1]
    def forward(self, x):
        return x.view(-1,65536)

In [41]:
n_classes = 10
model = torch.nn.Sequential(nn.Linear(512,512*8*8*2),
                            #nn.Linear(512*16,512*16*16),
                            Reshape(),
            #nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            ASPP(simclr_model.module.n_features, 64, 512),
                            Reshape1(),
            torch.nn.Linear(in_features=512*64*2,
                            out_features=512),
            torch.nn.Linear(in_features=512,
                            out_features=640000),
            torch.nn.Sigmoid())

model = nn.DataParallel(model)
print(model)

DataParallel(
  (module): Sequential(
    (0): Linear(in_features=512, out_features=65536, bias=True)
    (1): Reshape()
    (2): ASPP(
      (global_pooling): AdaptiveAvgPool2d(output_size=1)
      (relu): ReLU(inplace=True)
      (aspp1): Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (aspp2): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6), bias=False)
      (aspp3): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12), bias=False)
      (aspp4): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(18, 18), dilation=(18, 18), bias=False)
      (aspp5): Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (aspp1_bn): BatchNorm2d(64, eps=0.0003, momentum=0.1, affine=True, track_running_stats=True)
      (aspp2_bn): BatchNorm2d(64, eps=0.0003, momentum=0.1, affine=True, track_running_stats=True)
      (aspp3_bn): BatchNorm2d(64, eps=0.0003, momentum=0.1, affine=True, track_

In [42]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.BCELoss()

In [ ]:
for epoch in range(args.logistic_epochs):
    loss_epoch, accuracy_epoch = train(args, train_loader, simclr_model, model, criterion, optimizer)
    print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")

# final testing
loss_epoch, accuracy_epoch = test(args, test_loader, simclr_model, model, criterion, optimizer)
print(f"[FINAL]\t Loss: {loss_epoch / len(test_loader)}\t Accuracy: {accuracy_epoch / len(test_loader)}")

torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 

Step [27/441]	 Loss: 0.6954541206359863	 Accuracy: 0.26551997661590576
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])

...

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 1, 1])torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 320, 2, 2])torch.Size([64, 64, 1, 1])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 320, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 

torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])torch.Size([64, 64, 2, 2])

torch.Size([64, 64, 2, 2])
torch.Size([64, 320, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])torch.Size([64, 64, 2, 2])

torch.Size([64, 512, 2, 2])
*******
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 64, 2, 2])
torch.Size([64, 512, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 1, 1])
torch.Size([64, 

In [ ]:
torch.cuda.memory_stats()

In [ ]:
torch.cuda.max_memory_allocated(device=None)